In [41]:
import os
import pandas as pd
import numpy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime
from collections import OrderedDict
from copy import deepcopy 

# Here I test an simple idea:
We can buy stocks when the price above 30 day moving average and sell when price is below 30 day moving average.

Once the price of long stock is higher than the price that we long, we sell them.

In [8]:
dataset = pd.read_csv("data.csv")

In [13]:
dataset['mavg_last'] = dataset.groupby('ticker')['last'].transform(lambda x: x.rolling(30, 20).mean())
dataset['msd_last'] = dataset.groupby('ticker')['last'].transform(lambda x: x.rolling(30, 20).std())
dataset['mavg_volume'] = dataset.groupby('ticker')['volume'].transform(lambda x: x.rolling(30, 20).mean())
dataset['msd_volume'] = dataset.groupby('ticker')['volume'].transform(lambda x: x.rolling(30, 20).std())

In [17]:
# create a dictionary of daily tradable stocks (for simulation purpose)
all_dates = dataset['date'].drop_duplicates().tolist()

In [45]:
all_dates[-20]

'2021-02-19'

In [19]:
dataset[dataset['date'] == all_dates[0]]

,ticker,date,last,volume,mavg_last,mavg_volume,msd_last,msd_volume
0,1332 JT,2013-01-04,169.0987,1464100,NaN,NaN,NaN,NaN
3706,1334 JT,2013-01-04,147.8887,1553000,NaN,NaN,NaN,NaN
4006,1605 JT,2013-01-04,970.6780,5312400,NaN,NaN,NaN,NaN
6011,1721 JT,2013-01-04,950.5521,816600,NaN,NaN,NaN,NaN
8016,1801 JT,2013-01-04,1232.0186,2254000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
441444,9681 JT,2013-01-04,567.3170,1114500,NaN,NaN,NaN,NaN
443093,9735 JT,2013-01-04,3719.2396,800500,NaN,NaN,NaN,NaN
445098,9766 JT,2013-01-04,1721.7623,1513200,NaN,NaN,NaN,NaN
447103,9983 JT,2013-01-04,20584.8363,587300,NaN,NaN,NaN,NaN


In [34]:
dataset['shift_last'] = dataset['last'].shift(1)
dataset['shift_volume'] = dataset['volume'].shift(1)

In [33]:
dataset

,ticker,date,last,volume,mavg_last,mavg_volume,msd_last,msd_volume,shift_last
0,1332 JT,2013-01-04,169.0987,1464100,NaN,NaN,NaN,NaN,NaN
1,1332 JT,2013-01-07,166.3266,1783500,NaN,NaN,NaN,NaN,169.0987
2,1332 JT,2013-01-08,166.3266,1759800,NaN,NaN,NaN,NaN,166.3266
3,1332 JT,2013-01-09,165.4026,767800,NaN,NaN,NaN,NaN,166.3266
4,1332 JT,2013-01-10,167.2507,1503100,NaN,NaN,NaN,NaN,165.4026
...,...,...,...,...,...,...,...,...,...
451108,9984 JT,2021-03-15,10370.0000,12041200,9864.033333,1.388603e+07,700.116812,4.450800e+06,10635.0000
451109,9984 JT,2021-03-16,10620.0000,11346000,9947.766667,1.379312e+07,629.510959,4.474490e+06,10370.0000
451110,9984 JT,2021-03-17,10400.0000,9771000,10012.500000,1.374615e+07,567.864587,4.510121e+06,10620.0000
451111,9984 JT,2021-03-18,10220.0000,13941600,10067.833333,1.381674e+07,498.033449,4.495545e+06,10400.0000


In [ ]:
def generate_signal(datapoint, with_position):
    # return 1 if we shall buy
    # return -1 if we can sell
    if(datapoint['last'] <= datapoint['price_lower_bound'] and (datapoint['volume'] >= datapoint['volume_upper_bound'] or with_position)):
        return 1
    elif(datapoint['last'] >= datapoint['price_upper_bound'] and (datapoint['volume'] >= datapoint['volume_upper_bound'] or with_position) ):
        return -1
    else 0
    
    

In [ ]:
status = {'date': '', 'available_fund': '', 'portfolio': {'AA': {'trade_price':0, 'position':0}}}

In [38]:
def check_position(last_status, ticker):
    if(last_status['date'] == ''):
        return False
    else:
        if(ticker in last_status['portfolio']):
            return True
        else:
            return False
        

In [128]:
def run_test(annual_initial_investment, start_date = datetime.datetime(2013,1,4), end_date = datetime.datetime.today(), price_sigma_multiplier = 1.5, volume_sigma_multiplier = 1.5, rolling_window = 60, min_obs = 20, selected_stock_max = 20):
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")    
    used_data = dataset[(dataset['date'] >= start_date_str) & (dataset['date'] <= end_date_str)]
    last_date = used_data['date'].tolist()[-1]
    last_10_date = used_data['date'].tolist()[-10]
    
    used_data['mavg_last'] = used_data.groupby('ticker')['shift_last'].transform(lambda x: x.rolling(rolling_window, min_obs).mean())
    used_data['msd_last'] = used_data.groupby('ticker')['shift_last'].transform(lambda x: x.rolling(rolling_window, min_obs).std())
    used_data['mavg_volume'] = used_data.groupby('ticker')['shift_volume'].transform(lambda x: x.rolling(rolling_window, min_obs).mean())
    used_data['msd_volume'] = used_data.groupby('ticker')['shift_volume'].transform(lambda x: x.rolling(rolling_window, min_obs).std())
    
    used_data = used_data.dropna(subset = ['mavg_last', 'msd_last','mavg_volume', 'msd_volume'])
    
    #calcuilate statistics used to generate signal
    used_data['volume_upper_bound'] = used_data.apply(lambda x: x['mavg_volume'] + volume_sigma_multiplier*x['msd_volume'], axis = 1)
    used_data['volume_lower_bound'] = used_data.apply(lambda x: x['mavg_volume'] - volume_sigma_multiplier*x['msd_volume'], axis = 1)
    used_data['price_upper_bound'] = used_data.apply(lambda x: x['mavg_last'] + price_sigma_multiplier*x['msd_last'], axis = 1)
    used_data['price_lower_bound'] = used_data.apply(lambda x: x['mavg_last'] - price_sigma_multiplier*x['msd_last'], axis = 1)
    
    status = OrderedDict()
    for current_date in used_data['date'].tolist():
        daily_cross_section = used_data[used_data['date'] == current_date]
        
        if(current_date == last_date):
            # in the last trade date, clear the position
            last_status = status[next(reversed(status))]
            stocks_with_positions = new_status['portfolio']
            new_status = deepcopy(last_status)
            new_status['date'] = current_date
            new_positions = {}
            PnL_history = []
            for ticker in stocks_with_positions:
                stock = stocks_with_positions[ticker]
                if(len(daily_cross_section[daily_cross_section['ticker'] == ticker]) == 0):
                    continue
                current_price = daily_cross_section[daily_cross_section['ticker'] == ticker].to_dict("record")[0]['last']
                if(stock['position'] > 0 ):
                    temp_PnL = {'ticker': ticker, 'date': current_date, 'position': stock['position'], 'long_at': stock['trade_price'], 'clear_at': current_price,
                               'PnL': stock['position'] * (current_price - stock['trade_price']), 'return': (current_price - stock['trade_price'])/stock['trade_price']}
                    PnL_history.append(temp_PnL)
                    new_status['available_fund'] = new_status['available_fund'] + stock['position']*current_price
                elif(stock['position'] < 0):
                    temp_PnL = {'ticker': ticker, 'date': current_date, 'position': stock['position'], 'short_at': stock['trade_price'], 'clear_at': current_price,
                               'PnL': -stock['position'] * (stock['trade_price'] -current_price ), 'return': (stock['trade_price'] - current_price)/current_price}
                    PnL_history.append(temp_PnL)
                    new_status['available_fund'] = new_status['available_fund'] + temp_PnL['PnL']
                else:
                    new_positions[ticker] = stocks_with_positions[ticker]

            new_status['portfolio'] = new_positions
            new_status['PnL_history'] = PnL_history
            status[current_date] = new_status
            summary_obj = {'initial_investment':annual_initial_investment,'start_date': start_date, 'end_date': end_date,
                          'trading_history': status, 'final_available_funds': new_status['available_fund'], 'final_amount': new_status['available_fund']+new_status['surplus'], 'PnL': new_status['available_fund']+new_status['surplus'] - annual_initial_investment,
                          'return': (new_status['available_fund']+new_status['surplus']-annual_initial_investment)/annual_initial_investment }
            return summary_obj
        
        if(len(status) == 0):
            last_status = {'date': '', 'available_fund': annual_initial_investment, 'portfolio': {}, 'surplus' :0}
        else:
            last_status = status[next(reversed(status))]
        new_status = deepcopy(last_status)
        new_status['date'] = current_date
        
#         print(current_date, "pre clear", new_status['available_fund'])
        # clear position firstly
        stocks_with_positions = new_status['portfolio']
        new_positions = {}
        PnL_history = []
        total_PnL = 0
        for ticker in stocks_with_positions:
            stock = stocks_with_positions[ticker]
            if(len(daily_cross_section[daily_cross_section['ticker'] == ticker]) == 0):
                    continue
            current_price = daily_cross_section[daily_cross_section['ticker'] == ticker].to_dict("record")[0]['last']
            if(stock['position'] > 0 and stock['trade_price'] < current_price):
                temp_PnL = {'ticker': ticker, 'date': current_date, 'position': stock['position'], 'long_at': stock['trade_price'], 'clear_at': current_price,
                           'PnL': stock['position'] * (current_price - stock['trade_price']), 'return': (current_price - stock['trade_price'])/stock['trade_price']}
                PnL_history.append(temp_PnL)
                new_status['available_fund'] = new_status['available_fund'] + stock['position']*current_price
                total_PnL = total_PnL + temp_PnL['PnL']
            elif(stock['position'] < 0 and stock['trade_price'] > current_price):
                temp_PnL = {'ticker': ticker, 'date': current_date, 'position': stock['position'], 'short_at': stock['trade_price'], 'clear_at': current_price,
                           'PnL': -stock['position'] * (stock['trade_price'] -current_price ), 'return': (stock['trade_price'] - current_price)/current_price}
                PnL_history.append(temp_PnL)
                new_status['available_fund'] = new_status['available_fund'] + temp_PnL['PnL']
                total_PnL = total_PnL + temp_PnL['PnL']
            else:
                new_positions[ticker] = stocks_with_positions[ticker]
        
#         print(current_date, "after clear", new_status['available_fund'])
        
        surplus = total_PnL
        new_status['surplus'] =new_status['surplus'] + surplus/2
        new_status['available_fund'] = new_status['available_fund']-surplus/2
        stocks_with_position = list(new_positions.keys())
        mask = daily_cross_section['ticker'].isin(stocks_with_position)
        daily_cross_section_left = daily_cross_section[~mask]
        
        if(current_date <= last_10_date):

            # long
            long_stocks = daily_cross_section_left[(daily_cross_section_left['last'] <= daily_cross_section_left['price_lower_bound']) & (daily_cross_section_left['volume'] >= daily_cross_section_left['volume_upper_bound'])]

            # short
            short_stocks = daily_cross_section_left[(daily_cross_section_left['last'] >= daily_cross_section_left['price_upper_bound']) & (daily_cross_section_left['volume'] >= daily_cross_section_left['volume_upper_bound'])]

            total_trade = len(long_stocks) + len(long_stocks)
#             print(len(long_stocks))
            if(total_trade > 0 ):
#                 print(new_status['available_fund'])


                if(last_status['date'] == ''):
                    each_stock_value = new_status['available_fund']/2/total_trade
                else:
                    each_stock_value = new_status['available_fund']/total_trade


                if(len(long_stocks) > 0 and new_status['available_fund'] > 0):
                    long_count = 0
                    for stock in long_stocks.sort_values(by=['mavg_last'],ascending = False).to_dict("record"):
                        if(long_count > selected_stock_max):
                            continue
                        new_positions[stock['ticker']] = {'trade_price':stock['last'], 'position':each_stock_value/stock['last']}
                        new_status['available_fund'] = new_status['available_fund'] - each_stock_value
                        long_count += 1
                        if(new_status['available_fund'] <=0 ):
                            break
                # disable short
                if(len(short_stocks) < 0):
                    short_count = 0
                    for stock in short_stocks.sort_values(by=['mavg_last'],ascending = True).to_dict("record"):
                        if(short_count > 0):
                            continue
                        new_positions[stock['ticker']] = {'trade_price':stock['last'], 'position':-each_stock_value/stock['last']}
                        long_count += 1
                        # in short, we just borrow instead of spending actual money
                        # just record the positions but don't change the available_fund

        new_status['portfolio'] = new_positions
        new_status['PnL_history'] = PnL_history
        status[current_date] = new_status
        

        
            
    
    
    

In [133]:
backtest_result = run_test(1000000, datetime.datetime(2013,1,4), datetime.datetime(2022,6,22), selected_stock_max = 60)

/var/folders/wk/t1h51zds0zz6k6rbxcszdkqw0000gn/T/ipykernel_14508/1194760196.py:123: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  for stock in long_stocks.sort_values(by=['mavg_last'],ascending = False).to_dict("record"):
/var/folders/wk/t1h51zds0zz6k6rbxcszdkqw0000gn/T/ipykernel_14508/1194760196.py:76: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  current_price = daily_cross_section[daily_cross_section['ticker'] == ticker].to_dict("record")[0]['last']
/var/folders/wk/t1h51zds0zz6k6rbxcszdkqw0000gn/T/ipykernel_14508/1194760196.py:37: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', '

Further developments:
1. Currently I run a default set of parameters. We can adjust the code to tune the parameters dynamically every year based on the back-test from the previous years
2. Optimize how to construct the portfolio
3. Although the code contains the function of short, I currently only run the part of long. The short part can be modified further


In [134]:
# The final PnL
print("The final PnL is ", backtest_result['PnL'])

The final PnL is  353619.17685121717


In [135]:
# The annual PnL
print("The annual PnL is ", backtest_result['PnL']/10)

# The annual return
print("The annual return is ", backtest_result['PnL']/10/1000000)


The annual PnL is  35361.91768512172
The annual return is  0.03536191768512172
